# Processamento dos slides

Primeiramente, a conversão é de PPTX para TXT. É possível pular essa etapa e ir diretamente para JSON, mas a conversão para TXT é útil para visualização e depuração.

In [21]:
from pptx import Presentation
import glob

files = glob.glob("slides_adapt\\*.pptx")
# files = ["slides\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO.pptx"]
# files.remove("slides\\01.COLETÂNEA_IGREJAS_2022_TV-16.9.pptx")
# files.remove("slides\\03.COLETÂNEA DE CIAS_2021 TV.pptx")
files

['slides_adapt\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx',
 'slides_adapt\\03.COLETÂNEA DE CIAS_2021 TV_ADAPT.pptx',
 'slides_adapt\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx',
 'slides_adapt\\LOUVORES AVULSOS_Rev_31.12.22_ADAPT.pptx']

In [22]:
from pptx.enum.shapes import MSO_SHAPE_TYPE

for file in files:
    new_file = "slides_txt\\" + file.split("\\")[1] + ".txt"

    f = open(new_file, "w")

    prs = Presentation(file)
    for i, slide in enumerate(prs.slides):

        f.write(f"\nSLIDE_{i}\n")

        for shape in slide.shapes:
            f.write(f"SHAPE_{shape.shape_type}")

            if shape.shape_type == MSO_SHAPE_TYPE.AUTO_SHAPE:
                f.write(f"_AUTO_SHAPE_{shape.auto_shape_type}\n")
                f.write(f"HEIGHT_{shape.height}\n")
                f.write(f"TOP_{shape.top}\n")

            if hasattr(shape, "text"):
                f.write(f"\nSTART_TEXT\n{shape.text}\nEND_TEXT\n")
            
            f.write("\n")

        for shape in slide.shapes:
            if hasattr(shape, "text"):
                if shape.text.lower() == "índice":
                    f.write("\n__END__\n")

    f.close()

# Processamento do texto

In [23]:
files_txt = glob.glob("slides_txt\\*.txt")
files_txt

['slides_txt\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx.txt',
 'slides_txt\\03.COLETÂNEA DE CIAS_2021 TV_ADAPT.pptx.txt',
 'slides_txt\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx.txt',
 'slides_txt\\LOUVORES AVULSOS_Rev_31.12.22_ADAPT.pptx.txt']

In [24]:
import re
import json

auto_shape_pattern = r"_AUTO_SHAPE_([A-Z_]+)"
shape_pattern = r"SHAPE_([A-Z_]+)"

for file_txt in files_txt:
    praises = []
    with open(file_txt, "r") as f:
        text = f.read()
    
    for praise in text.split("__END__"):
        praise_struc = {}
        praise_struc["slides"] = []

        for slide in praise.split("SLIDE_"):
            slide_struc = {}
            slide_num = slide.split("\n")[0].strip()
            if not slide_num:
                continue

            slide_struc["slide"] = slide_num
            slide_struc["shapes"] = []

            for shape in slide.split("\nSHAPE_"):
                record_line = False
                text_inside = ""
                shapes_struc = {}

                for line in shape.split("\n"):
                    if "SHAPE_" in line:
                        match = re.search(auto_shape_pattern, line)
                        auto_shape = match.group(1)

                        shapes_struc["auto_shape"] = auto_shape
                        shapes_struc["shape"] = "AUTO_SHAPE"

                    if "HEIGHT_" in line:
                        height = line.split("_")[1]
                        shapes_struc["height"] = int(height)
                    if "TOP_" in line:
                        top = line.split("_")[1]
                        shapes_struc["top"] = int(top)

                    if "END_TEXT" in line:
                        record_line = False

                    if record_line:
                        text_inside += line + "\n"
                    else:
                        if text_inside:
                            shapes_struc["text"] = text_inside.strip()

                    if "START_TEXT" in line:
                        text_inside = ""
                        record_line = True

                if shapes_struc:
                    slide_struc["shapes"].append(shapes_struc)

            praise_struc["slides"].append(slide_struc)

        # pegar nome/numero do hino antes de salvar
        praises.append(praise_struc)

    new_file = "slides_json\\" + file_txt.split("\\")[1] + ".json"
    with open(new_file, "w") as f:
        f.write(json.dumps(praises))


# Processamento do JSON

In [25]:
def get_texts(d):
    if isinstance(d, dict):
        for k, v in d.items():
            if k == "text":
                yield v
            else:
                yield from get_texts(v)
    elif isinstance(d, list):
        for v in d:
            yield from get_texts(v)

In [26]:
import json

with open("slides_json\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx.txt.json", "r") as f:
    praises = json.load(f)

praises

[{'slides': [{'slide': '0',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3600945,
      'top': 2276522,
      'text': 'MUITO EM BREVE O REI VEM,\n(SIM, JESUS VEM!)\nMUITO EM BREVE O REI VEM,\n(SIM, JESUS VEM!) \nMUITO EM BREVE O REI VEM,\nMEU SENHOR JESUS VEM.'},
     {'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 1323399,
      'top': 402490,
      'text': '01AV- MUITO EM BREVE \nO REI VEM'}]},
   {'slide': '1',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3978012,
      'top': 600225,
      'text': 'VEM DEPRESSA, \nPOIS ELE TE CHAMA. \nTE CONVIDA A PARTICIPAR \nDA FESTA, \nTUDO JÁ ESTÁ PREPARADO,                               \nMEU SENHOR JESUS VEM.'}]},
   {'slide': '2',
    'shapes': [{'auto_shape': 'RECTANGLE',
      'shape': 'AUTO_SHAPE',
      'height': 3170099,
      'top': 3488723,
      'text': 'VEM BUSCAR UM POVO      \nPREPARADO, O ESPÍRITO NOS TEM REVELADO, \nVIG

In [27]:
erros_conhecidos = [
    "ABENÇOA-NOS SENHOR, DERRAMA SOBRE NÓS TUA PAZ ABENÇOA-NOS SENHOR, DERRAMA SOBRE NÓS TEU AMOR.",  # indice extra
    "MEU JESUS, SALVADOR,  OUTRO IGUAL NÃO HÁ. TODOS OS DIAS QUERO LOUVAR  AS MARAVILHAS DE TEU AMOR. CONSOLO, ABRIGO,  FORÇA E REFÚGIO É O SENHOR. COM TODO O MEU SER,  COM TUDO O QUE SOU,  SEMPRE TE ADORAREI.",  # pedaco de louvor
    "PODES CLAMAR, PODES CHORAR, EU ESTAREI PRONTO PRA TE AJUDAR, E SE O CORAÇÃO DESFALECER, CONFIA EM MIM SOU JESUS  E TE FAÇO VENCER.",  # pedaco de louvor
    "AO CORDEIRO GLÓRIA E HONRA,  SALVOS NÃO CESSEIS DE DAR. GLÓRIA, HONRA SEMPRE A DEUS ENTOAREI!  AMÉM!",  # indice extra
]
TAGS_LITERAIS = [
    "TODOS",
    "M",
    "H",
    "T",
    "BIS",
    "VARÕES",
    "SERVAS",
]
TAGS_CONTROLE = [
    "ÍNDICE",
    "CORO (2X)",
    "\n\nCORO",
    "CORO\n",
    "1X",
    "2X",
    "3X",
    "4X",
    "()",
    "(TODOS)",
    "(M)",
    "(H)",
    "(T)",
    "(BIS)",
    "(VARÕES)",
    "(SERVAS)",
    "REPETIR O LOUVOR",
    "REPETIR 1ª ESTROFE",
    "FINAL:",
    "BIS NO FINAL",
    "IGREJA CRISTÃ MARANATA",
    "ATUALIZAÇÃO",
    "\nINSTRUMENTOS",
]

In [28]:
import re


def return_possible_title(texts):
    possible_title = [
        text
        for text in texts
        if all(opt not in text.upper() for opt in TAGS_CONTROLE)
        and text.strip() != ""
        and text.upper() not in TAGS_LITERAIS
    ]
    if possible_title:
        min_string = min(possible_title, key=len)
        title_index = texts.index(min_string)
        title = min_string.upper().replace("\n", " ")
        if title not in erros_conhecidos:
            return title, title_index
    return None, None


def set_text_clean(texts_wo_title):
    texts_clean = [line for line in texts_wo_title if line not in TAGS_LITERAIS]
    new_texts_clean = []
    for line in texts_clean:
        for tag in TAGS_CONTROLE:
            line = line.upper().replace(tag, "")
        line = line.strip()
        if line:
            new_texts_clean.append(line)
    texts_clean = new_texts_clean
    texts_clean = list(dict.fromkeys(texts_clean))
    texts_clean = " ".join(texts_clean)
    texts_clean = texts_clean.replace("\n", " ")
    # remove all double quotes
    texts_clean = texts_clean.replace("“", "")
    texts_clean = texts_clean.replace("”", "")
    texts_clean = texts_clean.replace('"', "")
    return texts_clean


def set_text_full(texts_wo_title):
    texts_full = [
        line
        for line in texts_wo_title
        if line.upper() != "ÍNDICE" and line.strip() != ""
    ]
    texts_full = "\n\n".join(texts_full)
    texts_full = texts_full.replace("\n\nBIS", "\nBIS")
    texts_full = texts_full.replace('"', "'")
    return texts_full


def return_number(title):
    if title:
        for word in title.split(" "):
            match = re.search(r"\d+", word)
            if match:
                return match.group()
    return "null"


def process_structure(praises):
    new_structure = []
    for praise in praises:
        texts = list(get_texts(praise))

        if not texts:
            continue

        # clean double or more spaces in string array
        texts = [re.sub(r"\s+", " ", text.replace("–", "-")) for text in texts]

        title, title_index = return_possible_title(texts)
        numero = return_number(title)

        texts_wo_title = texts.copy()
        if title_index is not None:
            texts_wo_title.pop(title_index)

    
        texts_full = set_text_full(texts_wo_title)
        texts_clean = set_text_clean(texts_wo_title)

        new_structure.append(
        {
            "numero": numero,
            "nome": title if title is not None else "null",
            "texto": texts_full,
            "texto_limpo": texts_clean,
        }
    )
        
    return new_structure

new_structure = process_structure(praises)

new_structure

[{'numero': '01',
  'nome': '01AV- MUITO EM BREVE O REI VEM',
  'texto': 'MUITO EM BREVE O REI VEM, (SIM, JESUS VEM!) MUITO EM BREVE O REI VEM, (SIM, JESUS VEM!) MUITO EM BREVE O REI VEM, MEU SENHOR JESUS VEM.\n\nVEM DEPRESSA, POIS ELE TE CHAMA. TE CONVIDA A PARTICIPAR DA FESTA, TUDO JÁ ESTÁ PREPARADO, MEU SENHOR JESUS VEM.\n\nVEM BUSCAR UM POVO PREPARADO, O ESPÍRITO NOS TEM REVELADO, VIGIAI, O REI VEM, ELE VEM.',
  'texto_limpo': 'MUITO EM BREVE O REI VEM, (SIM, JESUS VEM!) MUITO EM BREVE O REI VEM, (SIM, JESUS VEM!) MUITO EM BREVE O REI VEM, MEU SENHOR JESUS VEM. VEM DEPRESSA, POIS ELE TE CHAMA. TE CONVIDA A PARTICIPAR DA FESTA, TUDO JÁ ESTÁ PREPARADO, MEU SENHOR JESUS VEM. VEM BUSCAR UM POVO PREPARADO, O ESPÍRITO NOS TEM REVELADO, VIGIAI, O REI VEM, ELE VEM.'},
 {'numero': '02',
  'nome': '02AV - PREPARAI-VOS, Ó IGREJA AMADA',
  'texto': 'PREPARAI-VOS, Ó IGREJA AMADA, POIS O NOIVO EM BREVE VIRÁ! Ó NOIVA AMADA, ATAVIADA, PREPARADA PARA O NOIVO ESTÁ!\n\nCOM ALEGRIA, A PROCLAMAR: OH! J

# Criar arquivo de inserção

In [29]:
import glob

files_json = glob.glob("slides_json\\*.json")
files_json

['slides_json\\01.COLETANEA_IGREJAS_2022_TV-16.9_ADAPT.pptx.txt.json',
 'slides_json\\03.COLETÂNEA DE CIAS_2021 TV_ADAPT.pptx.txt.json',
 'slides_json\\Louvores Avulsos CIAs 2022 - COM ANIMAÇÃO_ADAPT.pptx.txt.json',
 'slides_json\\LOUVORES AVULSOS_Rev_31.12.22_ADAPT.pptx.txt.json']

In [30]:
for index, file in enumerate(files_json):
    file_name = (
        "00"
        + str(index + 3)
        + "-"
        + file.split("\\")[1].split("pptx")[0].lower().replace(" ", "_")
        + "sql"
    )

    with open(file, "r") as f:
        louvores = json.load(f)

    louvores_estruturados = process_structure(louvores)
    with open("db\\migrations\\" + file_name, "w") as f:
        for hino in louvores_estruturados:
            text = (
                "INSERT INTO hino (numero, nome, texto, texto_limpo, coletanea_id, date_insert, date_update) VALUES ('"
                + hino["numero"]
                + "', '"
                + hino["nome"]
                + "', '"
                + hino["texto"].replace("\n", "\\n").replace("'", "''")
                + "', '"
                + hino["texto_limpo"].replace("'", "''")
                + "', "
                + str(index + 1)
                + ", CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);\n"
            )
            f.write(text)